In [1]:
import matplotlib.pyplot as plt
from pyomo.environ import *
from pyomo.dae import *

import numpy as np
from scipy.interpolate import interp2d
import pandas as pd 
from itertools import permutations, product, combinations

from fim_doe import *
from pyomo.contrib.sensitivity_toolbox.sens import sipopt
from idaes.apps.uncertainty_propagation.sens import get_dsdp

    'pyomo.common.sorting.sorted_robust'.  Please update your import.
    (deprecated in 6.1) (called from <frozen importlib._bootstrap>:228)


## Define fixed bed model 

In [2]:
from fixed_bed_model_indexed_dynamic import *

alpha option:
1 / (1 + exp(-x))
    Import ComponentSet from pyomo.common.collections.  (deprecated in 5.7.1)
    (called from <frozen importlib._bootstrap>:228)


In [3]:
def add_model(doe, timepoints, time_start=0):
    '''
    add variables, equations and discretize the model time
    Argument:
        timepoints: the timesteps
        time_start: where the timesteps start. For self defined timepoints it's 0. For experimental data it's -10.
    '''
    add_variables(doe, timesteps=timepoints, start=time_start)
    
    add_equations(doe)
    
    print ('the number of timepoints is', NFEt)
    
def discretize(doe, no_points=68, collo=False):
    '''
    no_points: how many time invertals to divide. For self defined timepoints it's 69. For experimental data it's 68. 
    here it's 68
    '''
    if collo:
        TransformationFactory('dae.collocation').apply_to(doe, nfe=no_points, ncp=3, scheme='LAGRANGE-RADAU', wrt=doe.t)
    else:
        TransformationFactory('dae.finite_difference').apply_to(doe, nfe=no_points, scheme='BACKWARD', wrt=doe.t)
    
    
def initialize(doe, init_point):
    '''
    Argument: 
        init_point: initial point, csv file
    '''
    # Initialize 
    print('The init point is', init_point)    
    store = pd.read_csv(init_point)
    position_max = store['position'].max()
    store['position'] = store['position'] / position_max

    initial_bed_csv(doe, store)
    fix_initial_bed(doe)
    
def generate_exp(feed, bath, y):
    dv_dict_overall = {'temp_feed': {0: feed}, 'temp_bath': {0:bath}, 'yfeed': {0: y}}
    return dv_dict_overall

In [10]:
NFEt = 68
time_scale = 3200
collo = False
exp_time_ = time_points(NFEt, time_scale)

scena_all = {'scena-name': [0], 'fitted_transport_coefficient':212, 'ua': np.log(5.0E6)}

initialize_point = '/Users/wangjialu/dowlinglab/sorption-fixed-bed/20200729_energy.csv'
#init_point = '20210916_feed313_bath293.csv'
#init_point = '/Users/wangjialu/dowlinglab/sorption-fixed-bed/fixed_bed_initial/T353y20_5e3.csv'
#init_point = '20210916_feed293_bath313_5e4.csv'
#init_point = 'init_test.csv'
#init_point = 'T353y20_5e3.csv'

In [11]:
def model_integrate(scena, args_list, exp_time=exp_time_, init_point=initialize_point):
    test = create_model(scena, temp_feed=293.15, temp_bath=303.15, y=0.2, doe_model=args_list[0], k_aug=args_list[1], opt=False, diff=0)
    add_model(test, exp_time)
    discretize(test)
    initialize(test, init_point)
    return test


In [12]:
# choose from model and k_aug 
model_opt = 'doe'

if (model_opt == 'doe'):
    args_ = [True, False]
    
elif (model_opt =='kaug'):
    args_ = [True, True]
    

parameter_dict = {'fitted_transport_coefficient':212, 'ua': np.log(5.0E6)}

dv_pass = {'temp_feed': [0], 'temp_bath':[0], 'yfeed': [0]}

measurement = ['FCO2']
t_measure = exp_time_


In [13]:
# empty prior
prior_all = np.zeros((2,2))

prior_pass=np.asarray(prior_all)

#L_initials = np.linalg.cholesky(prior_pass)
#print(L_initials)

print('The prior information FIM:', prior_pass)
print('Prior Det:', np.linalg.det(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigvals(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigh(prior_pass)[1])

The prior information FIM: [[0. 0.]
 [0. 0.]]
Prior Det: 0.0
Eigenvalue of the prior experiments FIM: [0. 0.]
Eigenvalue of the prior experiments FIM: [[1. 0.]
 [0. 1.]]


In [14]:
# choose from 'simultaneous', 'sequential', 'sipopt'
#sensi_opt = 'simultaneous_finite'
sensi_opt = 'sequential_finite'
#sensi_opt = 'sequential_sipopt'
#sensi_opt = 'sequential_kaug'
#sensi_opt = 'direct_kaug'

# feed, bath, y
exp1 = generate_exp(313.15, 293.15, 0.15)

In [15]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measurement, t_measure, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None, args=args_)

if_s = True 

result = doe_object.compute_FIM(exp1, mode=sensi_opt, FIM_store_name = 'fixed_bed.csv',
                               scale_nominal_param_value = if_s, formula='central')

if sensi_opt == 'simultaneous_finite':
    result.extract_FIM(doe_object.m, doe_object.design_timeset,
                                                          doe_object.square_result, doe_object.obj_opt)
else: 
    result.calculate_FIM(doe_object.jac, doe_object.design_values)

Sensitivity information is scaled by its corresponding parameter nominal value.
This scenario: {'fitted_transport_coefficient': {0: 214.12}, 'ua': {0: 15.424948470398375}, 'jac-index': {'fitted_transport_coefficient': [0], 'ua': [0]}, 'eps-abs': {'fitted_transport_coefficient': 2.1412, 'ua': 0.15424948470398375}, 'scena-name': [0]}
The inlet feed density is 41.0275814575221 [mol/m3]
the number of timepoints is 68
The init point is /Users/wangjialu/dowlinglab/sorption-fixed-bed/20200729_energy.csv
Model # of time grid is 69 , initial point # of time grid is 69
Parameters:  214.12 , 15.424948470398375
temp_feed is fixed at  313.15
temp_bath is fixed at  293.15
yfeed is fixed at  0.15
Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For m

 183r 0.0000000e+00 2.05e+02 6.18e+09   0.9 9.85e-01   9.9 5.54e-04 2.34e-04h  1
 184r 0.0000000e+00 2.05e+02 6.16e+09   0.9 1.00e+00   9.4 3.40e-05 3.63e-03f  1
 185r 0.0000000e+00 2.05e+02 6.15e+09   0.9 9.93e-01   9.8 1.11e-03 9.15e-04h  1
 186r 0.0000000e+00 2.05e+02 5.87e+09   0.9 1.11e+00   9.4 1.41e-04 3.54e-02h  1
 187r 0.0000000e+00 2.05e+02 5.86e+09   0.9 1.01e+00   9.8 1.14e-02 1.54e-03h  1
 188r 0.0000000e+00 2.05e+02 9.04e+09   0.9 1.03e+00   9.3 2.27e-04 8.41e-02h  1
 189r 0.0000000e+00 2.05e+02 5.10e+09   0.9 5.01e+00   9.7 1.18e-04 5.30e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 0.0000000e+00 2.05e+02 5.08e+09   0.9 9.43e-01  10.2 1.40e-03 4.71e-03h  1
 191r 0.0000000e+00 2.05e+02 5.07e+09   0.9 9.39e-01  10.6 1.18e-03 1.51e-03h  1
 192r 0.0000000e+00 2.05e+02 1.20e+10   0.9 9.39e-01  10.1 7.32e-04 2.73e-02h  1
 193r 0.0000000e+00 2.05e+02 1.20e+10   0.9 9.13e-01  10.5 3.52e-03 1.59e-03h  1
 194r 0.0000000e+00 2.05e+02

 366r 0.0000000e+00 1.26e+02 1.00e+03   0.9 1.10e+00   3.0 1.00e+00 1.00e+00f  1
 367r 0.0000000e+00 1.26e+02 1.00e+03   0.9 4.12e-01   3.4 1.00e+00 1.00e+00f  1
 368r 0.0000000e+00 1.24e+02 1.00e+03   0.9 1.24e+00   2.9 1.00e+00 1.00e+00f  1
 369r 0.0000000e+00 1.24e+02 1.00e+03   0.9 4.64e-01   3.3 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 370r 0.0000000e+00 1.23e+02 1.00e+03   0.9 1.39e+00   2.9 1.00e+00 1.00e+00f  1
 371r 0.0000000e+00 1.22e+02 1.00e+03   0.9 5.22e-01   3.3 1.00e+00 1.00e+00f  1
 372r 0.0000000e+00 1.20e+02 1.00e+03   0.9 1.57e+00   2.8 1.00e+00 1.00e+00f  1
 373r 0.0000000e+00 1.20e+02 1.00e+03   0.9 5.87e-01   3.2 1.00e+00 1.00e+00f  1
 374r 0.0000000e+00 1.18e+02 1.09e+03   0.9 1.76e+00   2.8 1.00e+00 1.00e+00f  1
 375r 0.0000000e+00 1.17e+02 1.00e+03   0.9 6.61e-01   3.2 1.00e+00 1.00e+00f  1
 376r 0.0000000e+00 1.16e+02 1.33e+03   0.9 1.98e+00   2.7 7.63e-01 6.83e-01f  1
 377r 0.0000000e+00 1.15e+02

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 550r 0.0000000e+00 2.93e-02 8.99e+01  -2.8 1.27e-01    -  4.26e-01 2.80e-01f  1
 551r 0.0000000e+00 1.84e-02 1.76e+02  -2.8 1.15e-01    -  2.17e-01 5.25e-01f  1
 552r 0.0000000e+00 1.42e-02 1.27e+02  -2.8 9.58e-02    -  3.01e-01 2.73e-01f  1
 553r 0.0000000e+00 1.03e-02 1.88e+02  -2.8 8.58e-02    -  6.71e-01 3.49e-01f  1
 554r 0.0000000e+00 6.78e-03 1.29e+02  -2.8 7.59e-02    -  2.46e-01 4.95e-01f  1
 555r 0.0000000e+00 5.08e-03 1.02e+02  -2.8 6.67e-02    -  2.79e-01 3.28e-01f  1
 556r 0.0000000e+00 3.99e-03 8.57e+01  -2.8 5.91e-02    -  4.08e-01 2.69e-01f  1
 557r 0.0000000e+00 2.79e-03 1.03e+02  -2.8 5.11e-02    -  5.71e-01 4.22e-01f  1
 558r 0.0000000e+00 1.96e-03 5.43e+01  -2.8 4.44e-02    -  4.27e-01 4.54e-01f  1
 559r 0.0000000e+00 1.40e-03 8.40e+01  -2.8 3.54e-02    -  5.20e-01 3.84e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 560r 0.0000000e+00 1.08e-03

 187r 0.0000000e+00 1.67e-03 9.26e+01  -2.7 3.86e-02    -  6.51e-01 4.92e-01f  1
 188r 0.0000000e+00 9.92e-04 4.53e+01  -2.7 2.83e-02    -  6.33e-01 5.62e-01f  1
 189r 0.0000000e+00 5.19e-04 5.54e+01  -2.7 1.79e-02    -  7.52e-01 5.89e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 0.0000000e+00 2.87e-05 6.84e+00  -2.7 5.05e-03    -  1.00e+00 1.00e+00f  1
 191r 0.0000000e+00 1.25e-05 1.12e+00  -2.7 1.58e-03    -  1.00e+00 1.00e+00h  1
 192r 0.0000000e+00 1.22e-05 2.11e-02  -2.7 1.02e-04    -  1.00e+00 1.00e+00h  1
 193r 0.0000000e+00 5.20e-04 3.83e+02  -4.0 8.53e-02    -  1.00e+00 2.42e-01f  1
 194r 0.0000000e+00 7.58e-04 1.51e+02  -4.0 2.89e-02    -  1.00e+00 7.57e-01f  1
 195r 0.0000000e+00 2.92e-06 8.55e+00  -4.0 1.42e-03    -  9.52e-01 1.00e+00f  1
 196r 0.0000000e+00 5.33e-08 2.48e-02  -4.0 2.30e-04    -  1.00e+00 1.00e+00h  1
 197r 0.0000000e+00 9.68e-09 7.12e-08  -4.0 1.61e-07    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 1

 185r 0.0000000e+00 7.40e-01 1.46e+02  -0.5 4.71e-01   0.8 1.00e+00 1.00e+00f  1
 186r 0.0000000e+00 2.59e-01 1.13e+02  -0.5 2.80e-01   1.2 1.00e+00 1.00e+00f  1
 187r 0.0000000e+00 7.73e-01 2.83e+02  -0.5 5.24e-01   0.8 1.00e+00 7.77e-01f  1
 188r 0.0000000e+00 1.85e-01 1.04e+02  -0.5 1.96e-01   1.2 1.00e+00 1.00e+00f  1
 189r 0.0000000e+00 5.36e-01 1.89e+02  -0.5 5.83e-01   0.7 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 0.0000000e+00 1.68e+00 3.59e+02  -0.5 6.46e+01    -  2.64e-01 1.85e-01f  1
 191r 0.0000000e+00 1.39e+00 1.15e+03  -0.5 2.84e+00   0.2 5.36e-01 1.42e-01f  1
 192r 0.0000000e+00 1.45e+00 7.40e+02  -0.5 3.97e+00  -0.2 1.00e+00 4.56e-01f  1
 193r 0.0000000e+00 1.27e+00 6.15e+02  -0.5 5.27e+01    -  5.93e-01 1.27e-01f  1
 194r 0.0000000e+00 1.02e+00 1.96e+02  -0.5 4.67e+01    -  1.00e+00 1.00e+00f  1
 195r 0.0000000e+00 3.33e-01 7.12e+01  -0.5 3.95e-01    -  1.00e+00 1.00e+00f  1
 196r 0.0000000e+00 8.96e-02

Parameters:  212 , 15.270698985694391
temp_feed is fixed at  313.15
temp_bath is fixed at  293.15
yfeed is fixed at  0.15
Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes

 187r 0.0000000e+00 1.93e-03 1.96e+02  -2.9 4.74e-02    -  3.06e-01 3.46e-01f  1
 188r 0.0000000e+00 1.53e-03 1.42e+02  -2.9 4.08e-02    -  3.45e-01 2.95e-01f  1
 189r 0.0000000e+00 1.20e-03 8.78e+01  -2.9 3.66e-02    -  2.91e-01 3.21e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 0.0000000e+00 9.60e-04 6.68e+01  -2.9 3.02e-02    -  2.74e-01 2.52e-01f  1
 191r 0.0000000e+00 7.63e-04 1.15e+02  -2.9 2.29e-02    -  3.31e-01 2.38e-01f  1
 192r 0.0000000e+00 4.71e-04 7.48e+01  -2.9 1.14e-02    -  4.70e-01 4.15e-01f  1
 193r 0.0000000e+00 2.53e-06 2.35e+00  -2.9 8.09e-04    -  9.91e-01 1.00e+00f  1
 194r 0.0000000e+00 2.29e-06 8.67e-01  -2.9 7.07e-04    -  1.00e+00 1.00e+00h  1
 195r 0.0000000e+00 2.33e-06 1.39e-03  -2.9 2.87e-05    -  1.00e+00 1.00e+00h  1
 196r 0.0000000e+00 2.12e-04 2.76e+02  -6.5 2.93e-02    -  9.97e-01 4.47e-01f  1
 197r 0.0000000e+00 5.84e-05 6.66e+01  -6.5 4.18e-03    -  9.83e-01 8.16e-01f  1
 198r 0.0000000e+00 3.11e-06

In [16]:
print('======Result summary======')
print('Four design criteria log10() value:')
print('A-optimality:', np.log10(result.trace))
print('D-optimality:', np.log10(result.det))
print('E-optimality:', np.log10(result.min_eig))
print('Modified E-optimality:', np.log10(result.cond))

======Result summary======
Four design criteria log10() value:
A-optimality: -2.7551920649041515
D-optimality: -11.516577872520761
E-optimality: -8.761385379471282
Modified E-optimality: 6.0061928864218
